# Data Analysis - Exercise 4
This lecture is about the time series forecasting utilizing deep learning. We will discuss the natural gas consumption forecasting topic using provided dataset as in the previous lecture.

Raw dataset is available at [vsb.ai](https://vsb.ai/natural-gas-forecasting), but we will use already pre-processed version of it. You can download the simplified version from our [Github](https://github.com/rasvob/2020-2021-DA4).

[Open in Google colab](https://colab.research.google.com/github/rasvob/2020-2021-DA4/blob/master/04_Forecasting_ANN.ipynb)

[Download from Github](https://github.com/rasvob/2020-2021-DA4/blob/master/04_Forecasting_ANN.ipynb)

## We start with importing commonly used libraries. 
- We will use maily Plotly again (you can see [this link](https://plotly.com/python/plotly-express/) for more information) for our visualizations.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
# from plotly.offline import init_notebook_mode
# init_notebook_mode(connected=False)
pd.set_option('display.max_colwidth', 100)
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler
import datetime

We need to install the rstl dependency from pip for the time series decomposition.

In [ ]:
!pip install rstl
import rstl

In [ ]:
import tensorflow.compat.v2 as tf #use tensorflow v2 as a main 
import tensorflow.keras as keras # required for high level applications
tf.version.VERSION

In [ ]:
!pip install tqdm
from tqdm.notebook import trange, tqdm
tqdm.pandas()

We have prepared common metrics for model evaluation beforehand. We will use these functions later.

In [ ]:
"""
Computes MAPE
"""
def mean_absolute_percentage_error(y_true: np.array, y_pred: np.array) -> float:
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

"""
Computes SMAPE
"""
def symetric_mean_absolute_percentage_error(y_true: np.array, y_pred: np.array) -> float:
    return np.mean(np.abs((y_pred - y_true) / ((np.abs(y_true) + np.abs(y_pred))/2.0))) * 100

"""
Computes MAE, MSE, MAPE, SMAPE, R2
"""
def compute_metrics(df: pd.DataFrame) -> pd.DataFrame:
    y_true, y_pred = df['y_true'].values, df['y_pred'].values
    return compute_metrics_raw(y_true, y_pred)

def compute_metrics_raw(y_true: pd.Series, y_pred: pd.Series) -> pd.DataFrame:
    mae, mse, mape, smape, r2 = mean_absolute_error(y_true=y_true, y_pred=y_pred), mean_squared_error(y_true=y_true, y_pred=y_pred), mean_absolute_percentage_error(y_true=y_true, y_pred=y_pred), symetric_mean_absolute_percentage_error(y_true=y_true, y_pred=y_pred), r2_score(y_true=y_true, y_pred=y_pred)
    return pd.DataFrame.from_records([{'MAE': mae, 'MSE': mse, 'MAPE': mape, 'SMAPE': smape, 'R2': r2}], index=[0])

In [ ]:
def show_history(history):
    plt.figure()
    for key in history.history.keys():
        plt.plot(history.epoch, history.history[key], label=key)
    plt.legend()
    plt.tight_layout()

# There is nothing new here yet :)
## The start of the lecture is exactly the same as in the previous one.

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/rasvob/2020-2021-DA4/master/datasets/ppnet_metar_v8_MAD.csv', sep=';', index_col=0)

### We will drop year 2019 for now and use only years 2013 to 2018

In [ ]:
df = df[df.Year < 2019].copy()

In [ ]:
df.loc[:, 'TestSet'] = 0
df.loc[df.Year == 2018, 'TestSet'] = 1

Dataset covers six whole years from January 1, 2013 to December 31, 2018. All data features are available at an hourly frequency. The whole dataset is composed of 52,584 data points. These data points were assembled from three main components.

The first component was created from consumption data. Prague is the capital city of the Czech Republic and its distribution network consisted of 422,926 customers in 2018. Total consumption was 3.82 billion m3. Residential sector included 381,914 households (33.3% of consumption). Industrial sector consisted from 177 big (24.8% of consumption), 39,175 medium (18.9% of consumption) and 1,652 small customers (21.9% of consumption). Missing remainder to 100% were operational losses that occurred during distribution, e.g., pipeline leak. The heating season in the Czech Republic is from September 1 to May 31. Usually, it is required for the heating season to begin that the temperature drops below +13 °C for two consecutive days, and no warming is forecasted for the following days. The heating season usually represents about 70% - 75% of the whole year's consumption.

The second component includes weather variables. We have used data from the Prague LKPR airport weather station. Airports are required to periodically issue METAR (aerodrome routine meteorological report) information. Those reports are archived and preserved for a long time.

The third component representing economic features are natural gas price data. We have obtained price data from the Czech energy regulation office and included them in the dataset.

In [ ]:
df.head()

### We have multiple features in the dataset. Consumption is the forecasted endogenous variable, other features are treated as exogenous.

In [ ]:
df.columns

We have circa 52k datapoints which should be sufficient even for very complex models.

In [ ]:
df.shape

## It's not out of the question to look at the data one more time, it has been before a whole week after all...

In [ ]:
px.line(y=df['Consumption'], x=df.index, color=df.Year)

In [ ]:
df.index = pd.DatetimeIndex(df.index)

# We will start with the simplest task at hand - next hour consumption forecasting. 

## You will most commonly deal with RNN (various forms - LSTM, GRU and other modifications) or CNN in the time series data domain nowadays.
- There are of course use cases where classical MLP with fixed window is better, although it's usually in the univariate time series covering rather short time periods cases. You need to consider in these situations if you really need neural net or simple AR(p) model is enough.

## The problem framing is basically the same as in the ML approach we utilized in the last lecture. We need to transform the time series forecasting into a supervised learning task.
- We will use fixed-size time window for the feature engineering again.
- We will start with data of the last 24 hours combined with chosen time-related features as well as some dummy ones.

# Our first model will be univariate one - we will use only consumption data for simplicity.
- We will dive into more details lately - we will just train the baseline model now.

In [ ]:
X, y = pd.DataFrame(), df.Consumption.copy()
for x in trange(24, 1, -1):
    X.loc[:, f'Consumption_lag_{x}'] = df.Consumption.shift(x)
    
X_train, X_test, y_train, y_test = X[df.TestSet == 0], X[df.TestSet == 1], y[df.TestSet == 0], y[df.TestSet == 1]

In [ ]:
X_train_selected_features_nona = X_train.drop(['Residual_diff_from_midnight', 'Year', 'Month', 'Day', 'Hour', 'Day_of_week','Temperature', 'Pressure', 'Pressure2', 'Humidity', 'Wind_direction', 'Wind_speed', 'Phenomena', 'Recent_phenomena', 'Visibility', 'Dewpoint', 'Datetime.1', 'Clouds_low_text', 'Clouds_low_m', 'Clouds_medium_text', 'Clouds_medium_m', 'Clouds_high_text', 'Clouds_high_m', 'Residual', 'ResetSignal', 'IsMissing', 'Cena_bfill', 'TestSet'], axis=1, errors='ignore').dropna()
X_test_selected_features_nona = X_test.drop(['Residual_diff_from_midnight', 'Year', 'Month', 'Day', 'Hour', 'Day_of_week','Temperature', 'Pressure', 'Pressure2', 'Humidity', 'Wind_direction', 'Wind_speed', 'Phenomena', 'Recent_phenomena', 'Visibility', 'Dewpoint', 'Datetime.1', 'Clouds_low_text', 'Clouds_low_m', 'Clouds_medium_text', 'Clouds_medium_m', 'Clouds_high_text', 'Clouds_high_m', 'Residual', 'ResetSignal', 'IsMissing', 'Cena_bfill', 'TestSet'], axis=1, errors='ignore').dropna()
y_train_no_na = y_train.dropna()
y_test_no_na = y_test.dropna()
y_test_no_na_idx = y_test_no_na.index
y_train_no_na = y_train_no_na[y_train_no_na.index.isin(X_train_selected_features_nona.index)]
X_test_selected_features_nona = X_test_selected_features_nona[X_test_selected_features_nona.index.isin(y_test_no_na.index)]
X_train_selected_features_nona.shape, y_train_no_na.shape, X_test_selected_features_nona.shape, y_test_no_na.shape

In [ ]:
X_train_selected_features_nona = X_train_selected_features_nona.values.reshape(X_train_selected_features_nona.shape[0], 1, X_train_selected_features_nona.shape[1])
X_test_selected_features_nona = X_test_selected_features_nona.values.reshape(X_test_selected_features_nona.shape[0], 1, X_test_selected_features_nona.shape[1])
X_train_selected_features_nona.shape

In [ ]:
inp = keras.layers.Input(shape=(X_train_selected_features_nona.shape[1], X_train_selected_features_nona.shape[2]))
x = keras.layers.LSTM(units=64, activation='tanh', return_sequences=True)(inp)
x = keras.layers.Dense(64)(x)
x = keras.layers.Dropout(0.2)(x)
x = keras.layers.Dense(32)(x)
output_layer = keras.layers.Dense(units=1, activation='linear')(x)

model = keras.Model(inp, output_layer)

model.summary()

model.compile(loss=keras.losses.MeanSquaredError(), optimizer=keras.optimizers.Adam(), metrics=['mae'])

## Now we can train the model and evaluate the performance.

In [ ]:
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='weights.best.hdf5',
    save_weights_only=True,
    monitor='val_loss',
    mode='auto',
    save_best_only=True)

batch_size = 128
history = model.fit(X_train_selected_features_nona, y_train_no_na, epochs=10, batch_size=batch_size, validation_split=0.2, shuffle=False, callbacks=[model_checkpoint_callback])

In [ ]:
show_history(history)

In [ ]:
y_pred = model.predict(X_test_selected_features_nona).ravel()

In [ ]:
y_pred[:10]

In [ ]:
df_res = pd.DataFrame({'y_true': df.Consumption.copy()[y_test_no_na.index], 'y_pred': y_pred.ravel()}, index=y_test_no_na_idx)

In [ ]:
compute_metrics(df_res)

In [ ]:
df_res_s = df_res.stack().reset_index().rename({'level_1': 'Type', 0: 'Value'}, axis=1)

In [ ]:
px.line(df_res_s, y='Value', x='Datetime', color='Type')

# We can do definitely do better with using all the available features.
- We will difference the data by 24 hours to reduce time-dependency again and add exogenous variables.

In [ ]:
X, y = pd.DataFrame(), df.Consumption.copy().diff(24)
for x in trange(24, 1, -1):
    X.loc[:, f'Consumption_lag_{x}'] = df.Consumption.shift(x)
    X.loc[:, f'Consumption_diff_lag_{x}'] = y.shift(x)
    X.loc[:, f'Temperature_lag_{x}'] = df['Temperature'].shift(x)
    X.loc[:, f'Humidity_lag_{x}'] = df['Humidity'].shift(x)
    X.loc[:, f'Cena_lag_{x}'] = df['Cena_bfill'].shift(x)
    
timestamp_s = df.index.map(datetime.datetime.timestamp)
hour = 60*60
day = 24*60*60
week = 7*day
month = (30.4369)*day
year = (365.2425)*day

X['Day_sin'] = np.sin(timestamp_s * (2 * np.pi / day))
X['Day_cos'] = np.cos(timestamp_s * (2 * np.pi / day))
X['Month_sin'] = np.sin(timestamp_s * (2 * np.pi / month))
X['Month_cos'] = np.cos(timestamp_s * (2 * np.pi / month))
X['Week_sin'] = np.sin(timestamp_s * (2 * np.pi / week))
X['Week_cos'] = np.cos(timestamp_s * (2 * np.pi / week))

X['IsSummer'] = 0
X.loc[df.Month.between(6, 8), 'IsSummer'] = 1
X['IsHeatingSeason'] = 1
X.loc[df.Month.between(6, 8), 'IsHeatingSeason'] = 0
X['Temperature'] = df.Temperature
heat_final = X.apply(lambda x: 1 if x['Temperature'] < 13 and x['IsHeatingSeason'] == 1 else 0, axis=1)
X['IsHeatingSeason'] = heat_final
X = X.drop(['Temperature'], axis=1)
X['IsWeekend'] = 0
X.loc[df.Day_of_week.between(6, 7), 'IsWeekend'] = 1
X_train, X_test, y_train, y_test = X[df.TestSet == 0], X[df.TestSet == 1], y[df.TestSet == 0], y[df.TestSet == 1]

In [ ]:
X_train_selected_features_nona = X_train.drop(['Residual_diff_from_midnight', 'Year', 'Month', 'Day', 'Hour', 'Day_of_week','Temperature', 'Pressure', 'Pressure2', 'Humidity', 'Wind_direction', 'Wind_speed', 'Phenomena', 'Recent_phenomena', 'Visibility', 'Dewpoint', 'Datetime.1', 'Clouds_low_text', 'Clouds_low_m', 'Clouds_medium_text', 'Clouds_medium_m', 'Clouds_high_text', 'Clouds_high_m', 'Residual', 'ResetSignal', 'IsMissing', 'Cena_bfill', 'TestSet'], axis=1, errors='ignore').dropna()
X_test_selected_features_nona = X_test.drop(['Residual_diff_from_midnight', 'Year', 'Month', 'Day', 'Hour', 'Day_of_week','Temperature', 'Pressure', 'Pressure2', 'Humidity', 'Wind_direction', 'Wind_speed', 'Phenomena', 'Recent_phenomena', 'Visibility', 'Dewpoint', 'Datetime.1', 'Clouds_low_text', 'Clouds_low_m', 'Clouds_medium_text', 'Clouds_medium_m', 'Clouds_high_text', 'Clouds_high_m', 'Residual', 'ResetSignal', 'IsMissing', 'Cena_bfill', 'TestSet'], axis=1, errors='ignore').dropna()
y_train_no_na = y_train.dropna()
y_test_no_na = y_test.dropna()
y_test_no_na_idx = y_test_no_na.index
y_train_no_na = y_train_no_na[y_train_no_na.index.isin(X_train_selected_features_nona.index)]
X_test_selected_features_nona = X_test_selected_features_nona[X_test_selected_features_nona.index.isin(y_test_no_na.index)]
X_train_selected_features_nona.shape, y_train_no_na.shape, X_test_selected_features_nona.shape, y_test_no_na.shape

## The LSTM benefit if the data are rescaled into the (-1, 1) interval (Do you remember the *tanh* activation function output range?) because there may emerge convergence issues otherwise.

In [ ]:
scaler_X = MinMaxScaler(feature_range=(-1, 1)).fit(X_train_selected_features_nona)
X_train_selected_features_nona = scaler_X.transform(X_train_selected_features_nona)
X_test_selected_features_nona = scaler_X.transform(X_test_selected_features_nona)

## Now we can try LSTM model
- The LSTM network expects the input data (X) to be provided with a specific array structure in the form of: [samples, time steps, features].
- We can transform the prepared train and test input data into the expected structure using numpy.reshape()

## Beware that there are two ways of reshaping the data into the [samples, time steps, features] format.
- The first one is perhaps more natural. The exogenous variables together with the consumption data are provided as parallel time series. There is although inconvience with including other dummy variables as they are not time series itself. The feature engineering of using only some lagged variables is problematic as well. The network needs to learn all the patterns from the raw data which leads to very problematic convergence.
- The second approach treats row in the previously engineered dataset as one timestep, we just need to reshape the data.

### Original shape

In [ ]:
X_test_selected_features_nona.shape

### Transformed shape in [samples, time steps, features] format

In [ ]:
X_train_selected_features_nona = X_train_selected_features_nona.reshape(X_train_selected_features_nona.shape[0], 1, X_train_selected_features_nona.shape[1])
X_test_selected_features_nona = X_test_selected_features_nona.reshape(X_test_selected_features_nona.shape[0], 1, X_test_selected_features_nona.shape[1])
X_train_selected_features_nona.shape

In [ ]:
inp = keras.layers.Input(shape=(X_train_selected_features_nona.shape[1], X_train_selected_features_nona.shape[2]))
x = keras.layers.LSTM(units=64, activation='tanh', return_sequences=True)(inp)
x = keras.layers.GRU(units=32, activation='tanh', return_sequences=False)(x)
x = keras.layers.Dense(64)(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dropout(0.2)(x)
x = keras.layers.Dense(32)(x)
output_layer = keras.layers.Dense(units=1, activation='linear')(x)

model = keras.Model(inp, output_layer)

model.summary()

model.compile(loss=keras.losses.MeanSquaredError(), optimizer=keras.optimizers.Adam(), metrics=['mae'])

## Now we can train the model and evaluate the performance.

In [ ]:
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='weights.best.hdf5',
    save_weights_only=True,
    monitor='val_loss',
    mode='auto',
    save_best_only=True)

batch_size = 128
history = model.fit(X_train_selected_features_nona, y_train_no_na, epochs=25, batch_size=batch_size, validation_split=0.2, shuffle=False, callbacks=[model_checkpoint_callback])

In [ ]:
show_history(history)

In [ ]:
y_pred = model.predict(X_test_selected_features_nona).ravel()

In [ ]:
y_pred[:10]

In [ ]:
y_pred = y_pred + df.Consumption.copy().shift(24)[y_test_no_na.index]

In [ ]:
df_res = pd.DataFrame({'y_true': df.Consumption.copy()[y_test_no_na.index], 'y_pred': y_pred.ravel()}, index=y_test_no_na_idx)

In [ ]:
compute_metrics(df_res)

In [ ]:
df_res_s = df_res.stack().reset_index().rename({'level_1': 'Type', 0: 'Value'}, axis=1)

In [ ]:
px.line(df_res_s, y='Value', x='Datetime', color='Type')

# We will continue with CNN model
- The preprocessing is very similiar to the LSTM one - we need to reshape our data into the [samples, time_steps, features].
- 1D Convolution layers will then extract features (together with Pooling ones) from the time-series and we can combine them using the Dense layers and make a forecast.

In [ ]:
X, y = pd.DataFrame(), df.Consumption.copy().diff(24)
n_steps = 24
n_features = 4
for x in trange(n_steps+1, 1, -1):
    X.loc[:, f'Consumption_lag_{x}'] = df.Consumption.shift(x)
    X.loc[:, f'Consumption_diff_lag_{x}'] = y.shift(x)
    X.loc[:, f'Temperature_lag_{x}'] = df['Temperature'].shift(x)
    X.loc[:, f'Humidity_lag_{x}'] = df['Humidity'].shift(x)
    
timestamp_s = df.index.map(datetime.datetime.timestamp)
hour = 60*60
day = 24*60*60
week = 7*day
month = (30.4369)*day
year = (365.2425)*day

X_dummy = pd.DataFrame(index=df.index)
X_dummy['Day_sin'] = np.sin(timestamp_s * (2 * np.pi / day))
X_dummy['Day_cos'] = np.cos(timestamp_s * (2 * np.pi / day))
X_dummy['Month_sin'] = np.sin(timestamp_s * (2 * np.pi / month))
X_dummy['Month_cos'] = np.cos(timestamp_s * (2 * np.pi / month))
X_dummy['Week_sin'] = np.sin(timestamp_s * (2 * np.pi / week))
X_dummy['Week_cos'] = np.cos(timestamp_s * (2 * np.pi / week))

X_dummy.loc[:, 'IsSummer'] = 0
X_dummy.loc[df.Month.between(6, 8), 'IsSummer'] = 1
X_dummy.loc[:, 'IsHeatingSeason'] = 1
X_dummy.loc[df.Month.between(6, 8), 'IsHeatingSeason'] = 0
X_dummy.loc[:, 'Temperature'] = df.Temperature
heat_final = X_dummy.apply(lambda x: 1 if x['Temperature'] < 13 and x['IsHeatingSeason'] == 1 else 0, axis=1)
X_dummy.loc[:, 'IsHeatingSeason'] = heat_final
X_dummy = X_dummy.drop('Temperature', axis=1)
X_dummy.loc[:, 'IsWeekend'] = 0
X_dummy.loc[df.Day_of_week.between(6, 7), 'IsWeekend'] = 1
X_train, X_test, y_train, y_test = X[df.TestSet == 0], X[df.TestSet == 1], y[df.TestSet == 0], y[df.TestSet == 1]
X_train_dummy, X_test_dummy = X_dummy[df.TestSet == 0], X_dummy[df.TestSet == 1]

In [ ]:
X_train_selected_features_nona = X_train.drop(['Residual_diff_from_midnight', 'Year', 'Month', 'Day', 'Hour', 'Day_of_week','Temperature', 'Pressure', 'Pressure2', 'Humidity', 'Wind_direction', 'Wind_speed', 'Phenomena', 'Recent_phenomena', 'Visibility', 'Dewpoint', 'Datetime.1', 'Clouds_low_text', 'Clouds_low_m', 'Clouds_medium_text', 'Clouds_medium_m', 'Clouds_high_text', 'Clouds_high_m', 'Residual', 'ResetSignal', 'IsMissing', 'Cena_bfill', 'TestSet'], axis=1, errors='ignore').dropna()
X_test_selected_features_nona = X_test.drop(['Residual_diff_from_midnight', 'Year', 'Month', 'Day', 'Hour', 'Day_of_week','Temperature', 'Pressure', 'Pressure2', 'Humidity', 'Wind_direction', 'Wind_speed', 'Phenomena', 'Recent_phenomena', 'Visibility', 'Dewpoint', 'Datetime.1', 'Clouds_low_text', 'Clouds_low_m', 'Clouds_medium_text', 'Clouds_medium_m', 'Clouds_high_text', 'Clouds_high_m', 'Residual', 'ResetSignal', 'IsMissing', 'Cena_bfill', 'TestSet'], axis=1, errors='ignore').dropna()
y_train_no_na = y_train.dropna()
y_test_no_na = y_test.dropna()
y_test_no_na_idx = y_test_no_na.index
y_train_no_na = y_train_no_na[y_train_no_na.index.isin(X_train_selected_features_nona.index)]
X_test_selected_features_nona = X_test_selected_features_nona[X_test_selected_features_nona.index.isin(y_test_no_na.index)]
X_test_dummy = X_test_dummy[X_test_dummy.index.isin(y_test_no_na.index)]
X_train_dummy = X_train_dummy[X_train_dummy.index.isin(y_train_no_na.index)]

X_train_selected_features_nona.shape, y_train_no_na.shape, X_test_selected_features_nona.shape, y_test_no_na.shape, X_train_dummy.shape, X_test_dummy.shape

### Original shape

In [ ]:
X_test_selected_features_nona.shape

### Transformed shape in [samples, time steps, features] format

In [ ]:
X_train_selected_features_nona = X_train_selected_features_nona.values.reshape(X_train_selected_features_nona.shape[0], n_steps, n_features)
X_test_selected_features_nona = X_test_selected_features_nona.values.reshape(X_test_selected_features_nona.shape[0], n_steps, n_features)
X_train_selected_features_nona.shape

# Non-time series data needs to be included through separate Input layer and mixed with extracted features later in the model. There is no point in including these features in the Conv layer - there are no signal-related characteristics to extract.

In [ ]:
inp = keras.layers.Input(shape=(X_train_selected_features_nona.shape[1], X_train_selected_features_nona.shape[2]))
dummy_inp = keras.layers.Input(shape=(X_dummy.shape[1]))


x = keras.layers.Conv1D(filters=32, kernel_size=3, activation='relu')(inp)
x = keras.layers.MaxPool1D(pool_size=2)(x)
x = keras.layers.Flatten()(x)

cnc = keras.layers.concatenate([x, dummy_inp])

x = keras.layers.Dense(64)(cnc)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dropout(0.2)(x)
x = keras.layers.Dense(32)(x)
output_layer = keras.layers.Dense(units=1, activation='linear')(x)

model = keras.Model([inp, dummy_inp], output_layer)

model.summary()

model.compile(loss=keras.losses.MeanSquaredError(), optimizer=keras.optimizers.Adam(), metrics=['mae'])

## Now we can train the model and evaluate the performance.

In [ ]:
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='weights.best.hdf5',
    save_weights_only=True,
    monitor='val_loss',
    mode='auto',
    save_best_only=True)

batch_size = 128
history = model.fit([X_train_selected_features_nona, X_train_dummy], y_train_no_na, epochs=25, batch_size=batch_size, validation_split=0.2, shuffle=False, callbacks=[model_checkpoint_callback])

In [ ]:
show_history(history)

In [ ]:
y_pred = model.predict([X_test_selected_features_nona, X_test_dummy]).ravel()

In [ ]:
y_pred[:10]

In [ ]:
y_pred = y_pred + df.Consumption.copy().shift(24)[y_test_no_na.index]

In [ ]:
df_res = pd.DataFrame({'y_true': df.Consumption.copy()[y_test_no_na.index], 'y_pred': y_pred.ravel()}, index=y_test_no_na_idx)

In [ ]:
compute_metrics(df_res)

In [ ]:
df_res_s = df_res.stack().reset_index().rename({'level_1': 'Type', 0: 'Value'}, axis=1)

In [ ]:
px.line(df_res_s, y='Value', x='Datetime', color='Type')

# Finally we can mix CNN and LSTM models together.
## We will use the [TimeDistributed](https://www.tensorflow.org/api_docs/python/tf/keras/layers/TimeDistributed) layers in the model. 

- Consider a batch of 32 video samples, where each sample is a 128x128 RGB image with channels_last data format, across 10 timesteps. The batch input shape is (32, 10, 128, 128, 3).
- You can then use TimeDistributed to apply the same Conv2D layer to each of the 10 timesteps, independently:
- **Because TimeDistributed applies the same instance of Conv2D to each of the timestamps, the same set of weights are used at each timestamp.**

In [ ]:
X, y = pd.DataFrame(), df.Consumption.copy().diff(24)
n_steps = 24
n_features = 4
for x in trange(n_steps+1, 1, -1):
    X.loc[:, f'Consumption_lag_{x}'] = df.Consumption.shift(x)
    X.loc[:, f'Consumption_diff_lag_{x}'] = y.shift(x)
    X.loc[:, f'Temperature_lag_{x}'] = df['Temperature'].shift(x)
    X.loc[:, f'Humidity_lag_{x}'] = df['Humidity'].shift(x)
    
timestamp_s = df.index.map(datetime.datetime.timestamp)
hour = 60*60
day = 24*60*60
week = 7*day
month = (30.4369)*day
year = (365.2425)*day

X_dummy = pd.DataFrame(index=df.index)
X_dummy['Day_sin'] = np.sin(timestamp_s * (2 * np.pi / day))
X_dummy['Day_cos'] = np.cos(timestamp_s * (2 * np.pi / day))
X_dummy['Month_sin'] = np.sin(timestamp_s * (2 * np.pi / month))
X_dummy['Month_cos'] = np.cos(timestamp_s * (2 * np.pi / month))
X_dummy['Week_sin'] = np.sin(timestamp_s * (2 * np.pi / week))
X_dummy['Week_cos'] = np.cos(timestamp_s * (2 * np.pi / week))

X_dummy.loc[:, 'IsSummer'] = 0
X_dummy.loc[df.Month.between(6, 8), 'IsSummer'] = 1
X_dummy.loc[:, 'IsHeatingSeason'] = 1
X_dummy.loc[df.Month.between(6, 8), 'IsHeatingSeason'] = 0
X_dummy.loc[:, 'Temperature'] = df.Temperature
heat_final = X_dummy.apply(lambda x: 1 if x['Temperature'] < 13 and x['IsHeatingSeason'] == 1 else 0, axis=1)
X_dummy.loc[:, 'IsHeatingSeason'] = heat_final
X_dummy = X_dummy.drop('Temperature', axis=1)
X_dummy.loc[:, 'IsWeekend'] = 0
X_dummy.loc[df.Day_of_week.between(6, 7), 'IsWeekend'] = 1
X_train, X_test, y_train, y_test = X[df.TestSet == 0], X[df.TestSet == 1], y[df.TestSet == 0], y[df.TestSet == 1]
X_train_dummy, X_test_dummy = X_dummy[df.TestSet == 0], X_dummy[df.TestSet == 1]

In [ ]:
X_train_selected_features_nona = X_train.drop(['Residual_diff_from_midnight', 'Year', 'Month', 'Day', 'Hour', 'Day_of_week','Temperature', 'Pressure', 'Pressure2', 'Humidity', 'Wind_direction', 'Wind_speed', 'Phenomena', 'Recent_phenomena', 'Visibility', 'Dewpoint', 'Datetime.1', 'Clouds_low_text', 'Clouds_low_m', 'Clouds_medium_text', 'Clouds_medium_m', 'Clouds_high_text', 'Clouds_high_m', 'Residual', 'ResetSignal', 'IsMissing', 'Cena_bfill', 'TestSet'], axis=1, errors='ignore').dropna()
X_test_selected_features_nona = X_test.drop(['Residual_diff_from_midnight', 'Year', 'Month', 'Day', 'Hour', 'Day_of_week','Temperature', 'Pressure', 'Pressure2', 'Humidity', 'Wind_direction', 'Wind_speed', 'Phenomena', 'Recent_phenomena', 'Visibility', 'Dewpoint', 'Datetime.1', 'Clouds_low_text', 'Clouds_low_m', 'Clouds_medium_text', 'Clouds_medium_m', 'Clouds_high_text', 'Clouds_high_m', 'Residual', 'ResetSignal', 'IsMissing', 'Cena_bfill', 'TestSet'], axis=1, errors='ignore').dropna()
y_train_no_na = y_train.dropna()
y_test_no_na = y_test.dropna()
y_test_no_na_idx = y_test_no_na.index
y_train_no_na = y_train_no_na[y_train_no_na.index.isin(X_train_selected_features_nona.index)]
X_test_selected_features_nona = X_test_selected_features_nona[X_test_selected_features_nona.index.isin(y_test_no_na.index)]
X_test_dummy = X_test_dummy[X_test_dummy.index.isin(y_test_no_na.index)]
X_train_dummy = X_train_dummy[X_train_dummy.index.isin(y_train_no_na.index)]

X_train_selected_features_nona.shape, y_train_no_na.shape, X_test_selected_features_nona.shape, y_test_no_na.shape, X_train_dummy.shape, X_test_dummy.shape

# Beware that now we need to use format of [samples, subsequences, timesteps, features]!

In [ ]:
X_train_selected_features_nona = X_train_selected_features_nona.values.reshape(X_train_selected_features_nona.shape[0], 2, n_steps//2, n_features)
X_test_selected_features_nona = X_test_selected_features_nona.values.reshape(X_test_selected_features_nona.shape[0], 2, n_steps//2, n_features)
X_train_selected_features_nona.shape

In [ ]:
inp = keras.layers.Input(shape=(X_train_selected_features_nona.shape[1:]))

x = keras.layers.TimeDistributed(keras.layers.Conv1D(filters=32, kernel_size=3, activation='relu'))(inp)
x = keras.layers.TimeDistributed(keras.layers.MaxPool1D(pool_size=2))(x)
x = keras.layers.TimeDistributed(keras.layers.Flatten())(x)
x = keras.layers.Bidirectional(keras.layers.LSTM(units=160, activation='tanh', return_sequences=False))(x)
x = keras.layers.Dense(64)(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dropout(0.2)(x)
x = keras.layers.Dense(32)(x)
output_layer = keras.layers.Dense(units=1, activation='linear')(x)

model = keras.Model(inp, output_layer)

model.summary()

model.compile(loss=keras.losses.MeanSquaredError(), optimizer=keras.optimizers.Adam(), metrics=['mae'])

## Now we can train the model and evaluate the performance.

In [ ]:
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='weights.best.hdf5',
    save_weights_only=True,
    monitor='val_loss',
    mode='auto',
    save_best_only=True)

batch_size = 128
history = model.fit(X_train_selected_features_nona, y_train_no_na, epochs=25, batch_size=batch_size, validation_split=0.2, shuffle=False, callbacks=[model_checkpoint_callback])

In [ ]:
show_history(history)

In [ ]:
y_pred = model.predict(X_test_selected_features_nona).ravel()

In [ ]:
y_pred[:10]

In [ ]:
y_pred = y_pred + df.Consumption.copy().shift(24)[y_test_no_na.index]

In [ ]:
df_res = pd.DataFrame({'y_true': df.Consumption.copy()[y_test_no_na.index], 'y_pred': y_pred.ravel()}, index=y_test_no_na_idx)

In [ ]:
compute_metrics(df_res)

In [ ]:
df_res_s = df_res.stack().reset_index().rename({'level_1': 'Type', 0: 'Value'}, axis=1)

In [ ]:
px.line(df_res_s, y='Value', x='Datetime', color='Type')

# Bonus: How to forecast multiple timesteps in the future?
This task is much simpler than in the ML approach in the last lecture.

We have specified only a single neuron as an output in our models so far. It is no problem to use multiple output neurons and output vector instead of scalar value.

Number of output neurons is equal to number of timesteps you want to forecast.

- Alternatively, you can use [TimeDistributed](https://machinelearningmastery.com/timedistributed-layer-for-long-short-term-memory-networks-in-python/) layer with carefully designed network architecture.

## The forecast horizon of 24 hours starting at midnight will be used.

In [ ]:
X, y = pd.DataFrame(), df.Consumption.copy()
n_steps = 24
n_features = 3
for x in trange(n_steps+1, 0, -1):
    X.loc[:, f'Consumption_lag_{x}'] = df.Consumption.shift(x)
    X.loc[:, f'Temperature_lag_{x}'] = df['Temperature'].shift(x)
    X.loc[:, f'Humidity_lag_{x}'] = df['Humidity'].shift(x)
    
timestamp_s = df.index.map(datetime.datetime.timestamp)
hour = 60*60
day = 24*60*60
week = 7*day
month = (30.4369)*day
year = (365.2425)*day

X_dummy = pd.DataFrame(index=df.index)
X_dummy['Day_sin'] = np.sin(timestamp_s * (2 * np.pi / day))
X_dummy['Day_cos'] = np.cos(timestamp_s * (2 * np.pi / day))
X_dummy['Month_sin'] = np.sin(timestamp_s * (2 * np.pi / month))
X_dummy['Month_cos'] = np.cos(timestamp_s * (2 * np.pi / month))
X_dummy['Week_sin'] = np.sin(timestamp_s * (2 * np.pi / week))
X_dummy['Week_cos'] = np.cos(timestamp_s * (2 * np.pi / week))

X_dummy.loc[:, 'IsSummer'] = 0
X_dummy.loc[df.Month.between(6, 8), 'IsSummer'] = 1
X_dummy.loc[:, 'IsHeatingSeason'] = 1
X_dummy.loc[df.Month.between(6, 8), 'IsHeatingSeason'] = 0
X_dummy.loc[:, 'Temperature'] = df.Temperature
heat_final = X_dummy.apply(lambda x: 1 if x['Temperature'] < 13 and x['IsHeatingSeason'] == 1 else 0, axis=1)
X_dummy.loc[:, 'IsHeatingSeason'] = heat_final
X_dummy = X_dummy.drop('Temperature', axis=1)
X_dummy.loc[:, 'IsWeekend'] = 0
X_dummy.loc[df.Day_of_week.between(6, 7), 'IsWeekend'] = 1

y_vec = pd.DataFrame()
for x in range(1, 25):
    y_vec.loc[:, f'T+{x}'] = y.shift(-x)
    
X_train, X_test, y_train, y_test = X[df.TestSet == 0], X[df.TestSet == 1], y_vec[df.TestSet == 0], y_vec[df.TestSet == 1]
X_train_dummy, X_test_dummy = X_dummy[df.TestSet == 0], X_dummy[df.TestSet == 1]

In [ ]:
X_train_selected_features_nona = X_train.drop(['Residual_diff_from_midnight', 'Year', 'Month', 'Day', 'Hour', 'Day_of_week','Temperature', 'Pressure', 'Pressure2', 'Humidity', 'Wind_direction', 'Wind_speed', 'Phenomena', 'Recent_phenomena', 'Visibility', 'Dewpoint', 'Datetime.1', 'Clouds_low_text', 'Clouds_low_m', 'Clouds_medium_text', 'Clouds_medium_m', 'Clouds_high_text', 'Clouds_high_m', 'Residual', 'ResetSignal', 'IsMissing', 'Cena_bfill', 'TestSet'], axis=1, errors='ignore').dropna()
X_test_selected_features_nona = X_test.drop(['Residual_diff_from_midnight', 'Year', 'Month', 'Day', 'Hour', 'Day_of_week','Temperature', 'Pressure', 'Pressure2', 'Humidity', 'Wind_direction', 'Wind_speed', 'Phenomena', 'Recent_phenomena', 'Visibility', 'Dewpoint', 'Datetime.1', 'Clouds_low_text', 'Clouds_low_m', 'Clouds_medium_text', 'Clouds_medium_m', 'Clouds_high_text', 'Clouds_high_m', 'Residual', 'ResetSignal', 'IsMissing', 'Cena_bfill', 'TestSet'], axis=1, errors='ignore').dropna()
y_train_no_na = y_train.dropna()
y_test_no_na = y_test.dropna()
y_test_no_na_idx = y_test_no_na.index
y_train_no_na = y_train_no_na[y_train_no_na.index.isin(X_train_selected_features_nona.index)]
X_test_selected_features_nona = X_test_selected_features_nona[X_test_selected_features_nona.index.isin(y_test_no_na.index)]
X_test_dummy = X_test_dummy[X_test_dummy.index.isin(y_test_no_na.index)]
X_train_dummy = X_train_dummy[X_train_dummy.index.isin(y_train_no_na.index)]

X_train_selected_features_nona.shape, y_train_no_na.shape, X_test_selected_features_nona.shape, y_test_no_na.shape, X_train_dummy.shape, X_test_dummy.shape

### Original shape

In [ ]:
X_test_selected_features_nona.shape

### Transformed shape in [samples, time steps, features] format

In [ ]:
X_train_selected_features_nona = X_train_selected_features_nona.values.reshape(X_train_selected_features_nona.shape[0], n_steps+1, n_features)
X_test_selected_features_nona = X_test_selected_features_nona.values.reshape(X_test_selected_features_nona.shape[0], n_steps+1, n_features)
X_train_selected_features_nona.shape

In [ ]:
inp = keras.layers.Input(shape=(X_train_selected_features_nona.shape[1], X_train_selected_features_nona.shape[2]))
dummy_inp = keras.layers.Input(shape=(X_dummy.shape[1]))


x_1 = keras.layers.Conv1D(filters=64, kernel_size=5, activation='relu')(inp)
x_2 = keras.layers.Conv1D(filters=64, kernel_size=3, activation='relu')(inp)

x_mp1 = keras.layers.MaxPool1D(pool_size=2)(x_1)
x_mp2 = keras.layers.MaxPool1D(pool_size=2)(x_2)
x_flt_1 = keras.layers.Flatten()(x_mp1)
x_flt_2 = keras.layers.Flatten()(x_mp2)

cnc = keras.layers.concatenate([x_flt_1, x_flt_2, dummy_inp])

x = keras.layers.Dense(128)(cnc)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dropout(0.2)(x)
x = keras.layers.Dense(64)(x)
output_layer = keras.layers.Dense(units=y_train_no_na.shape[1], activation='linear')(x)

model = keras.Model([inp, dummy_inp], output_layer)

model.summary()

model.compile(loss=keras.losses.MeanSquaredError(), optimizer=keras.optimizers.Adam(), metrics=['mae'])

## Now we can train the model and evaluate the performance.

In [ ]:
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='weights.best.hdf5',
    save_weights_only=True,
    monitor='val_loss',
    mode='auto',
    save_best_only=True)

batch_size = 256
history = model.fit([X_train_selected_features_nona, X_train_dummy], y_train_no_na, epochs=50, batch_size=batch_size, validation_split=0.2, shuffle=False, callbacks=[model_checkpoint_callback])

In [ ]:
show_history(history)

In [ ]:
y_pred = model.predict([X_test_selected_features_nona, X_test_dummy])

In [ ]:
y_pred = pd.DataFrame(y_pred, index=pd.to_datetime(y_test_no_na.index), columns=y_vec.columns)

In [ ]:
y_pred.head()

## Now we need to extract only forecasts created at midnight

In [ ]:
y_pred = pd.Series(y_pred[y_pred.index.hour == 0].stack().values, index=y_test_no_na_idx)
y_pred.head()

In [ ]:
df_res = pd.DataFrame({'y_true': df.Consumption.copy()[y_test_no_na.index], 'y_pred': y_pred.ravel()}, index=y_test_no_na_idx)

In [ ]:
compute_metrics(df_res)

In [ ]:
df_res_s = df_res.stack().reset_index().rename({'level_1': 'Type', 0: 'Value'}, axis=1)

### We can see that the model more-or-less captured underlaying trend in the data
- The forecasts are very imprecise - it's rather flat
- The summer levels are totally incorrect

#### What do you propose for making the model better? Any ideas or techniques from the last lecture you would like to try?

In [ ]:
px.line(df_res_s, y='Value', x='Datetime', color='Type')

# Tasks for the rest of the lecture
1. Try to modify any of the proposed models or create a new one from scratch.
2. Optional - use any preprocessing technique you would like to try (STL, standardizing/normalizing inputs etc.) with any chosen model.
3. Compare the new model with the original one. Did the MAE, MSE etc changed? If it did, how (is it better or worse, do you have any idea why)?